In [112]:
import os, sys, time
import tifffile as tif
from scipy.ndimage.interpolation import zoom
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import csv
import IPython

In [113]:
test = np.load('/home/emilyjanedennis/Desktop/forpeter_PRA-new-to_WHS/posttransformed_zyx_voxels.npy')

# initialize
# in zyx
resample_dims = (455,774,302) #whs
original_dims = (865,1610,539) #largewhs
d1, d2 = test.shape
nx4centers = np.ones((d1, d2+1))
nx4centers[:, :-1] = test

# acount for resampling by creating transformmatrix
zr, yr, xr = resample_dims
z, y, x = original_dims

# apply scale diff
trnsfrmmatrix = np.identity(4)*(zr/float(z), yr/float(y), xr/float(x), 1)

# nx4 * 4x4 to give transform
trnsfmdpnts = nx4centers.dot(trnsfrmmatrix)  # z,y,x


In [114]:
#if you're in downloaded waxholm space (WHS_SD_rat_T2star_v1.01.nii)
# and you want to get bregma coordinates
# using info from jukebox/LightSheetData/brodyatlas/atlas/original/coordinates_v1-v1.01

#voxel coordinates in ITKsnap or FIJI coordinates
# z slices 1-512 range from ventral to dorsal (- to 0/+)
# x is ML 1-512 from animal's right (-) to left (+)
# y is 1-1024 AP from cerebellum to nose (- to +)
# original waxholm bregma 0 0 0 is at (x245,y653,z440) 
# each voxel is 0.0390625 mm

x,y,z =[400,900,340]
meas=0.0390625
AP = (y-653)*meas
ML = (x-245)*meas
DV = (z-440)*meas

print(AP,ML,DV)

9.6484375 6.0546875 -3.90625


In [115]:
trnsfmdpnts

array([[183.97469895, 157.64823731, 107.24809906,   1.        ],
       [283.23851783, 159.98507984,  92.98720018,   1.        ],
       [280.59540916, 179.74589597,  75.97439774,   1.        ],
       [262.77530417, 178.44197903, 115.12023389,   1.        ],
       [291.90303497, 180.63370377, 130.41132481,   1.        ],
       [230.38154972, 191.90627583,  81.37307524,   1.        ],
       [232.24437653, 193.12523978, 136.6989015 ,   1.        ],
       [232.30957934, 201.73047755, 135.96917201,   1.        ],
       [282.96146176, 202.34901989, 132.78745801,   1.        ],
       [252.83083154, 204.00128827, 104.76184961,   1.        ],
       [308.43220869, 221.18505841, 109.41864534,   1.        ],
       [324.91362319, 244.72306328,  74.68570266,   1.        ],
       [ 69.27435649, 323.19277984, 169.4708775 ,   1.        ],
       [249.83642499, 366.31463751,  27.45983858,   1.        ],
       [ 60.35826434, 388.2234719 , 162.12417   ,   1.        ],
       [374.13968289, 409

In [116]:
np.shape(trnsfmdpnts)

(18, 4)

In [172]:
#if you're in 'tiff space' -- 
# WHS_SD_rat_T2star_v1.01_atlas.tiff
# or WHS_SD_rat_atlas_v3_annotation.tiff
pt = 13
# enter your x,y,z coordinates from FIJI/ImageJ:
xt, yt, zt =[trnsfmdpnts[pt][2],trnsfmdpnts[pt][1],trnsfmdpnts[pt][0]]


# and you want to know where you are in original waxholm space

# z slices 1-455 are ML left(+) to right (-)
    # and cropped at 18 to 473 of original x1-512
# y is 1-774 AP nose to cerebellum (+ to -)
    # and cropped at  y192-948
# x is DV 1-302 dorsal to ventral (0/+ to -)
    # and cropped at z148-450

meas=0.0390625
x = 474-zt
#zt 1-455 == x473-19 because crop doesn't include 'first frame'
y = 949-yt
#yt 1-774 == y948-193
z = 450 - xt
#xt 1-302 == z450-149
print("coordinates in original waxholm space (x,y,z)")
print(x,y,z)

# or if you want to get bregma coordinates

AP = (296-yt)*meas
ML = (229-zt)*meas
DV = (10-xt)*meas
print("bregma coordinates (AP, ML, DV) **NOTE** DV is calculated from skull surface, add 0.899 to calculate from brain surface (this is done for you when making the image later on in this file)")
print(AP,ML,DV)

coordinates in original waxholm space (x,y,z)
224.16357501156068 582.685362489441 422.5401614211503
bregma coordinates (AP, ML, DV) **NOTE** DV is calculated from skull surface, add 0.899 to calculate from brain surface (this is done for you when making the image later on in this file)
-2.7466655277562113 -0.8139228511109109 -0.6820249444863173


In [173]:
# to do - make a user-friendly PRA conversion for mPRA and fPRA

In [174]:
#here's a nice image
# this is adjusted for brain surface (what paxinos watson uses) instead of skull surface (which waxholm uses)
linktosite = "http://labs.gaidi.ca/rat-brain-atlas/?ml={}&ap={}&dv={}".format(ML,AP,DV+0.899)
pt

13

In [175]:
IPython.display.IFrame(linktosite,width=800,height=800,embed=True)